In [3]:
import numpy as np
import keras
from keras.callbacks import EarlyStopping,ModelCheckpoint
import pickle
from sklearn.preprocessing import StandardScaler
from keras.utils.vis_utils import plot_model

from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, CuDNNLSTM, Multiply,Dropout
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K

In [5]:
address = 'D:\\data_mfcc\\'

In [6]:
def loadData(id):
    with open(address + f'y_train_{id}','rb') as f:
        y_train=np.array(pickle.load(f))
    with open(address+f'y_test_{id}','rb') as f:
        y_test=np.array(pickle.load(f))
    with open(address+ f'X_train_{id}','rb') as f:
        X_train = np.array(pickle.load(f))
    with open(address+f'X_test_{id}','rb') as f:
        X_test = np.array(pickle.load(f))

    scalers = {}
    for k in range(X_train.shape[2]):
        scalers[k] = StandardScaler()
        X_train[:, k, :] = scalers[k].fit_transform(X_train[:, k, :]) 

    for j in range(X_test.shape[2]):
        X_test[:, j, :] = scalers[j].transform(X_test[:, j, :])
    
    print(f'id : {id}')
    from sklearn.preprocessing import LabelBinarizer
    encoder = LabelBinarizer()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.fit_transform(y_test)

   
    return X_train,y_train,X_test,y_test

In [9]:
total_num_people=[]
for i in range(1,92):
    if i <10:
        total_num_people.append('100'+str(i))
    else:
        total_num_people.append('10'+str(i))

for i in total_num_people[:4]:
    if True:
        X_train=[]
        X_test=[]
        y_train=[]
        y_test=[]
        X_train,y_train,X_test,y_test=loadData(i)
        
    else:
        continue


    #load data here
    #input_shape = batch_size,time_steps,input_dim

    #batch_size = X_train.shape[0]   #automatically taken no need to specify
    time_steps = X_train.shape[1]
    input_dim = X_train.shape[2]



    hid_pre_acts = [16]
    hid_post_acts = [96]
    #dense1s = [16,32,64,96,128,256]
    #dense2s = [16,32,64,96,128,256]
    for hid_pre_act in hid_pre_acts:
        for hid_post_act in hid_post_acts:
            # Defined shared layers as global variables
            repeator = RepeatVector(time_steps) #500
            concatenator = Concatenate(axis=-1)
            densor = Dense(1, activation = "relu")
            activator = keras.layers.Softmax(axis=1) # We are using a custom softmax(axis = 1) loaded in this notebook
            dotor = Dot(axes = 1)

            #one step attention

            def one_step_attention(a,s_prev):

                s_prev = repeator(s_prev)
                concat = concatenator([a,s_prev])
                e = densor(concat)
                alphas = activator(e)
                context = dotor([alphas, a])

                return context
            
            post_activation_lstm_cell = CuDNNLSTM(hid_post_act,return_state=True)
            #output_layer = Dense(dense1,activation='relu')
            #onemore = Dense(dense2,activation='relu')
            fin_out = Dense(6,activation='softmax')

                # model

            def model(time_steps,input_dim,hid_post_act,hid_pre_act):

                X = Input(shape =(time_steps,input_dim))
                s0 = Input(shape=(hid_post_act,),name='s0')
                c0 = Input(shape=(hid_post_act,),name='c0')
                s=s0
                c=c0
                outputs=[]


                # Pre-LSTM 16
                a = CuDNNLSTM(hid_pre_act,return_sequences=True)(X)
                a = Dropout(0.4)(a)
                #6 is final output size
                for t in range(1):
                    context =one_step_attention(a,s)
                    s, _, c = post_activation_lstm_cell(context, initial_state=[s, c])
                    #out = output_layer(s)
                    #o = onemore(out)
                    k=fin_out(s)
                    outputs.append(k)


                model = Model([X, s0, c0], outputs)

                return model       

            model = model(time_steps,input_dim,hid_post_act,hid_pre_act)

            opt = Adam()
            model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
            #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
            print(X_train.shape)
            s0 = np.zeros((X_train.shape[0], hid_post_act))
            c0 = np.zeros((X_train.shape[0], hid_post_act))
            s0_t = np.zeros((X_test.shape[0], hid_post_act))
            c0_t = np.zeros((X_test.shape[0], hid_post_act))
            print(f'id : {i}')
            filePath =f'C:\\Users\\djaym7\\Desktop\\Github\\EmotionRecognition\\saved_mfcc_atten_unidir_models\\{i}.h5'
            checkpoint = ModelCheckpoint(filepath=filePath,monitor='val_acc',mode='max',save_best_only=True,verbose=1,save_weights_only=False)
              
            model.fit([X_train, s0, c0], y_train, epochs=70,validation_data=([X_test,s0_t,c0_t],y_test),callbacks=[checkpoint])


id : 1001
(7360, 499, 13)
id : 1001
Train on 7360 samples, validate on 82 samples
Epoch 1/70
7360/7360 [==============================] - 38s 5ms/step - loss: 1.6550 - acc: 0.2970 - val_loss: 1.7376 - val_acc: 0.1707

Epoch 00001: val_acc improved from -inf to 0.17073, saving model to C:\Users\djaym7\Desktop\Github\EmotionRecognition\saved_mfcc_atten_unidir_models\1001.h5


C:\Users\djaym7\Anaconda3\lib\site-packages\keras\engine\network.py:872: UserWarning: Layer cu_dnnlstm_177 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 's0_88:0' shape=(?, 96) dtype=float32>, <tf.Tensor 'c0_88:0' shape=(?, 96) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/70
7360/7360 [==============================] - 19s 3ms/step - loss: 1.4841 - acc: 0.3913 - val_loss: 1.6621 - val_acc: 0.2195

Epoch 00002: val_acc improved from 0.17073 to 0.21951, saving model to C:\Users\djaym7\Desktop\Github\EmotionRecognition\saved_mfcc_atten_unidir_models\1001.h5
Epoch 3/70
7360/7360 [==============================] - 19s 3ms/step - loss: 1.4251 - acc: 0.4194 - val_loss: 1.5671 - val_acc: 0.2805

Epoch 00003: val_acc improved from 0.21951 to 0.28049, saving model to C:\Users\djaym7\Desktop\Github\EmotionRecognition\saved_mfcc_atten_unidir_models\1001.h5
Epoch 4/70
7360/7360 [==============================] - 19s 3ms/step - loss: 1.3840 - acc: 0.4432 - val_loss: 1.6115 - val_acc: 0.2439

Epoch 00004: val_acc did not improve from 0.28049
Epoch 5/70
7360/7360 [==============================] - 19s 3ms/step - loss: 1.3540 - acc: 0.4565 - val_loss: 1.5822 - val_acc: 0.2805

Epoch 00005: val_acc improved from 0.28049 to 0.28049, saving model to C:\Users\djaym7

7360/7360 [==============================] - 18s 2ms/step - loss: 1.0927 - acc: 0.5735 - val_loss: 2.1059 - val_acc: 0.2439

Epoch 00042: val_acc did not improve from 0.32927
Epoch 43/70
7360/7360 [==============================] - 17s 2ms/step - loss: 1.0868 - acc: 0.5803 - val_loss: 2.0992 - val_acc: 0.2561

Epoch 00043: val_acc did not improve from 0.32927
Epoch 44/70
7360/7360 [==============================] - 18s 2ms/step - loss: 1.0830 - acc: 0.5834 - val_loss: 2.0024 - val_acc: 0.2805

Epoch 00044: val_acc did not improve from 0.32927
Epoch 45/70
7360/7360 [==============================] - 19s 3ms/step - loss: 1.0807 - acc: 0.5792 - val_loss: 1.9491 - val_acc: 0.2805

Epoch 00045: val_acc did not improve from 0.32927
Epoch 46/70
7360/7360 [==============================] - ETA: 0s - loss: 1.0728 - acc: 0.587 - 19s 3ms/step - loss: 1.0728 - acc: 0.5878 - val_loss: 1.8897 - val_acc: 0.3293

Epoch 00046: val_acc did not improve from 0.32927
Epoch 47/70
7360/7360 [================

C:\Users\djaym7\Anaconda3\lib\site-packages\keras\engine\network.py:872: UserWarning: Layer cu_dnnlstm_179 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 's0_89:0' shape=(?, 96) dtype=float32>, <tf.Tensor 'c0_89:0' shape=(?, 96) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


...

Epoch 2/70
7360/7360 [==============================] - 17s 2ms/step - loss: 1.4779 - acc: 0.3923 - val_loss: 1.3422 - val_acc: 0.5122

Epoch 00002: val_acc improved from 0.47561 to 0.51220, saving model to C:\Users\djaym7\Desktop\Github\EmotionRecognition\saved_mfcc_atten_unidir_models\1003.h5
Epoch 3/70
7360/7360 [==============================] - 17s 2ms/step - loss: 1.4238 - acc: 0.4212 - val_loss: 1.2803 - val_acc: 0.5122

Epoch 00003: val_acc improved from 0.51220 to 0.51220, saving model to C:\Users\djaym7\Desktop\Github\EmotionRecognition\saved_mfcc_atten_unidir_models\1003.h5
Epoch 4/70
7360/7360 [==============================] - 16s 2ms/step - loss: 1.3760 - acc: 0.4437 - val_loss: 1.3093 - val_acc: 0.4756

Epoch 00004: val_acc did not improve from 0.51220
Epoch 5/70
7360/7360 [==============================] - 16s 2ms/step - loss: 1.3509 - acc: 0.4607 - val_loss: 1.3456 - val_acc: 0.4878

Epoch 00005: val_acc did not improve from 0.51220
Epoch 6/70
7360/7360 [==============

7360/7360 [==============================] - 16s 2ms/step - loss: 1.0937 - acc: 0.5799 - val_loss: 1.1925 - val_acc: 0.5366

Epoch 00042: val_acc did not improve from 0.57317
Epoch 43/70
7360/7360 [==============================] - 17s 2ms/step - loss: 1.0868 - acc: 0.5742 - val_loss: 1.2881 - val_acc: 0.4756

Epoch 00043: val_acc did not improve from 0.57317
Epoch 44/70
7360/7360 [==============================] - 16s 2ms/step - loss: 1.0931 - acc: 0.5720 - val_loss: 1.1959 - val_acc: 0.5122

Epoch 00044: val_acc did not improve from 0.57317
Epoch 45/70
7360/7360 [==============================] - 17s 2ms/step - loss: 1.1051 - acc: 0.5727 - val_loss: 1.2590 - val_acc: 0.4756

Epoch 00045: val_acc did not improve from 0.57317
Epoch 46/70
7360/7360 [==============================] - 17s 2ms/step - loss: 1.0897 - acc: 0.5708 - val_loss: 1.3844 - val_acc: 0.4878

Epoch 00046: val_acc did not improve from 0.57317
Epoch 47/70
7360/7360 [==============================] - 17s 2ms/step - loss:

C:\Users\djaym7\Anaconda3\lib\site-packages\keras\engine\network.py:872: UserWarning: Layer cu_dnnlstm_183 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 's0_91:0' shape=(?, 96) dtype=float32>, <tf.Tensor 'c0_91:0' shape=(?, 96) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/70
7360/7360 [==============================] - 19s 3ms/step - loss: 1.4958 - acc: 0.3889 - val_loss: 1.4520 - val_acc: 0.4512

Epoch 00002: val_acc improved from 0.45122 to 0.45122, saving model to C:\Users\djaym7\Desktop\Github\EmotionRecognition\saved_mfcc_atten_unidir_models\1004.h5
Epoch 3/70
7360/7360 [==============================] - 17s 2ms/step - loss: 1.4397 - acc: 0.4238 - val_loss: 1.5471 - val_acc: 0.4390

Epoch 00003: val_acc did not improve from 0.45122
Epoch 4/70
7360/7360 [==============================] - 16s 2ms/step - loss: 1.3981 - acc: 0.4385 - val_loss: 1.5009 - val_acc: 0.3902

Epoch 00004: val_acc did not improve from 0.45122
Epoch 5/70
7360/7360 [==============================] - 16s 2ms/step - loss: 1.3678 - acc: 0.4523 - val_loss: 1.4900 - val_acc: 0.4024

Epoch 00005: val_acc did not improve from 0.45122
Epoch 6/70
7360/7360 [==============================] - 16s 2ms/step - loss: 1.3422 - acc: 0.4626 - val_loss: 1.4744 - val_acc: 0.4512

Epoch 0000

7360/7360 [==============================] - 20s 3ms/step - loss: 1.1046 - acc: 0.5671 - val_loss: 1.2870 - val_acc: 0.4878

Epoch 00043: val_acc did not improve from 0.53659
Epoch 44/70
7360/7360 [==============================] - 17s 2ms/step - loss: 1.0928 - acc: 0.5700 - val_loss: 1.4411 - val_acc: 0.4756

Epoch 00044: val_acc did not improve from 0.53659
Epoch 45/70
7360/7360 [==============================] - 17s 2ms/step - loss: 1.0899 - acc: 0.5723 - val_loss: 1.3336 - val_acc: 0.4756

Epoch 00045: val_acc did not improve from 0.53659
Epoch 46/70
7360/7360 [==============================] - 17s 2ms/step - loss: 1.0822 - acc: 0.5764 - val_loss: 1.4198 - val_acc: 0.4756

Epoch 00046: val_acc did not improve from 0.53659
Epoch 47/70
7360/7360 [==============================] - 17s 2ms/step - loss: 1.0895 - acc: 0.5795 - val_loss: 1.4505 - val_acc: 0.5122

Epoch 00047: val_acc did not improve from 0.53659
Epoch 48/70
7360/7360 [==============================] - 18s 2ms/step - loss: